In [17]:
%run "match_sample_files.ipynb"

import pandas as pd
import pathlib as pl
import collections as col

DRY_RUN = False

MANUALLY_CURATED_SOURCES = {
    "UW_WH": ("UW", "hgsvc"),
}

for sample_name, sample_obj in SAMPLES.items():
    if sample_name == sample_obj.alt:
        continue
    sample_files = FILES_EXIST_PER_SAMPLE[sample_name]
    
    for fn, sample_file in enumerate(sample_files, start=0):
        key = sample_file.read_type, sample_file.file_group
        sample_obj.sample_files[key].append(sample_file.data_rel_path)
        if fn == 0:
            sample_obj.source_dates[sample_file.file_group].append(
                sample_file.group_date
            )
            sample_obj.source_dates[sample_file.file_group].append(
                sample_file.file_date
            )
        else:
            sample_obj.source_dates[sample_file.file_group].append(
                sample_file.file_date
            )
            
    
    if sample_obj.hifi_complete:
        for (read_type, read_group), read_files in sample_obj.sample_files.items():
            if read_type != "hifi":
                continue
            source_dates = sample_obj.source_dates[read_group]
            try:
                source_date = str(int(source_dates[0]))
            except ValueError:
                source_date, _ = col.Counter(source_dates).most_common(1)[0]
                
            read_group_info = MERGED.loc[MERGED["file_group"] == read_group, ["data_source", "project"]]
            if read_group_info.empty:
                read_group_info = set(
                    ALL_FILES.loc[ALL_FILES["file_group"] == read_group, "curated_by"].values
                )
                data_source, project = MANUALLY_CURATED_SOURCES[read_group_info]
            else:
                data_source = set(read_group_info["data_source"].values).pop()
                project = set(read_group_info["project"].values).pop()
                
            if data_source == project:
                assert data_source.startswith("HGSVC") and project.startswith("HGSVC")
                project = "hgsvc"
                data_source = data_source[5:].upper()
                
            source_path = sample_obj.get_file_group_lca_path(read_files)
            verified_file_path = PATH_PREFIX.local.joinpath(
                DATA_ROOT, source_path
            )
            assert verified_file_path.is_dir()
            verified_file_name = f"{sample_obj.name}.{len(read_files)}-cells.verified"
            verified_read_files = sorted([fp.name for fp in read_files])
            if not DRY_RUN:
                with open(verified_file_path.joinpath(verified_file_name), "w") as dump:
                    _ = dump.write(f"# {TODAY}\n")
                    _ = dump.write("\n".join(verified_read_files))
                    
                fofn_name = f"{sample_obj.name}_{read_type}_fastq.{project}-{data_source}-{source_date}.fofn"
                
                fofn_path = LOCAL_SAMPLE_ROOT.joinpath(
                    sample_obj.name, fofn_name
                )
                with open(fofn_path, "w") as dump:
                    for rf in sorted(read_files):
                        assert str(rf).startswith("project-centric")
                        _ = dump.write(str(rf) + "\n")
                
    if sample_obj.ont_complete:
        for (read_type, read_group), read_files in sample_obj.sample_files.items():
            if read_type != "ont":
                continue
            source_dates = sample_obj.source_dates[read_group]
            try:
                source_date = str(int(source_dates[0]))
            except ValueError:
                source_date, _ = col.Counter(source_dates).most_common(1)[0]
            
            read_group_info = MERGED.loc[MERGED["file_group"] == read_group, ["data_source", "project"]]
            if read_group_info.empty:
                read_group_info = set(
                    ALL_FILES.loc[ALL_FILES["file_group"] == read_group, "curated_by"].values
                )
            else:
                data_source = set(read_group_info["data_source"].values).pop()
                project = set(read_group_info["project"].values).pop()

            if data_source == project:
                assert data_source.startswith("HGSVC") and project.startswith("HGSVC")
                project = "hgsvc"
                data_source = data_source[5:].upper()
               
            source_path = sample_obj.get_file_group_lca_path(read_files)
            verified_file_path = PATH_PREFIX.local.joinpath(
                DATA_ROOT, source_path
            )
            assert verified_file_path.is_dir()
            verified_file_name = f"{sample_obj.name}.{len(read_files)}-cells.verified"
            verified_read_files = sorted([fp.name for fp in read_files])
            if not DRY_RUN:
                with open(verified_file_path.joinpath(verified_file_name), "w") as dump:
                    _ = dump.write(f"# {TODAY}\n")
                    _ = dump.write("\n".join(verified_read_files))
                
                fofn_name = f"{sample_obj.name}_{read_type}_fastq.{project}-{data_source}-{source_date}.fofn"
                
                fofn_path = LOCAL_SAMPLE_ROOT.joinpath(
                    sample_obj.name, fofn_name
                )

                with open(fofn_path, "w") as dump:
                    for rf in sorted(read_files):
                        assert str(rf).startswith("project-centric")
                        _ = dump.write(str(rf) + "\n")
                



Skipping over 19320 - not enough data

Skipping over 19320 - not enough data

Skipping over 19320 - not enough data

Skipping over 19320 - not enough data


hgsvc JAX
/mounts/hilbert/project/projects/medbioinf/data/00_RESTRUCTURE/sample-centric/NA19239/NA19239_ont_fastq.hgsvc-JAX-2021.fofn
hgsvc UW
/mounts/hilbert/project/projects/medbioinf/data/00_RESTRUCTURE/sample-centric/HG00732/HG00732_hifi_fastq.hgsvc-UW-2019.fofn
hgsvc UW
/mounts/hilbert/project/projects/medbioinf/data/00_RESTRUCTURE/sample-centric/HG00732/HG00732_hifi_fastq.hgsvc-UW-2020.fofn
hgsvc UW
/mounts/hilbert/project/projects/medbioinf/data/00_RESTRUCTURE/sample-centric/HG00732/HG00732_ont_fastq.hgsvc-UW-2023.fofn
hgsvc UW
/mounts/hilbert/project/projects/medbioinf/data/00_RESTRUCTURE/sample-centric/HG00733/HG00733_hifi_fastq.hgsvc-UW-2019.fofn
hgsvc UW
/mounts/hilbert/project/projects/medbioinf/data/00_RESTRUCTURE/sample-centric/HG00733/HG00733_hifi_fastq.hgsvc-UW-2021.fofn
hgsvc UW
/mounts/hilbert/project/projects/medbioinf/data/00_RESTRUCTURE/sample-centric/HG00733/HG00733_ont_fastq.hgsvc-UW-2023.fofn
hgsvc JAX
/mounts/hilbert/project/projects/medbioinf/data/00_RESTRUCTU

hgsvc JAX
/mounts/hilbert/project/projects/medbioinf/data/00_RESTRUCTURE/sample-centric/HG02106/HG02106_ont_fastq.hgsvc-JAX-2021.fofn
hgsvc JAX
/mounts/hilbert/project/projects/medbioinf/data/00_RESTRUCTURE/sample-centric/HG00268/HG00268_hifi_fastq.hgsvc-JAX-2021.fofn
hgsvc JAX
/mounts/hilbert/project/projects/medbioinf/data/00_RESTRUCTURE/sample-centric/HG00268/HG00268_ont_fastq.hgsvc-JAX-2021.fofn
hgsvc UW
/mounts/hilbert/project/projects/medbioinf/data/00_RESTRUCTURE/sample-centric/HG02769/HG02769_hifi_fastq.hgsvc-UW-2021.fofn
hgsvc UW
/mounts/hilbert/project/projects/medbioinf/data/00_RESTRUCTURE/sample-centric/HG02769/HG02769_ont_fastq.hgsvc-UW-2023.fofn
hgsvc UW
/mounts/hilbert/project/projects/medbioinf/data/00_RESTRUCTURE/sample-centric/HG03452/HG03452_hifi_fastq.hgsvc-UW-2021.fofn
hgsvc UW
/mounts/hilbert/project/projects/medbioinf/data/00_RESTRUCTURE/sample-centric/HG03452/HG03452_ont_fastq.hgsvc-UW-2023.fofn
hgsvc UMIGS
/mounts/hilbert/project/projects/medbioinf/data/00_REST